In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from dateutil.parser import parse
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, TensorDataset
import matplotlib.pyplot as plt

In [2]:
###TODO: expand to full data, figure out how to visualize the difference in compounds or just train multiple neural networks on each compound, fix the size error idk why thats happening

In [3]:
filenames = ["LapAndWeather/LapWeather_Australian Grand Prix.csv","LapAndWeather/LapWeather_Austrian Grand Prix.csv","LapAndWeather/LapWeather_Azerbaijan Grand Prix.csv","LapAndWeather/LapWeather_Bahrain Grand Prix.csv","LapAndWeather/LapWeather_Belgian Grand Prix.csv","LapAndWeather/LapWeather_Brazilian Grand Prix.csv","LapAndWeather/LapWeather_British Grand Prix.csv","LapAndWeather/LapWeather_Canadian Grand Prix.csv", "LapAndWeather/LapWeather_Chinese Grand Prix.csv","LapAndWeather/LapWeather_French Grand Prix.csv","LapAndWeather/LapWeather_German Grand Prix.csv","LapAndWeather/LapWeather_Hungarian Grand Prix.csv","LapAndWeather/LapWeather_Italian Grand Prix.csv","LapAndWeather/LapWeather_Japanese Grand Prix.csv","LapAndWeather/LapWeather_Mexican Grand Prix.csv","LapAndWeather/LapWeather_Monaco Grand Prix.csv","LapAndWeather/LapWeather_Russian Grand Prix.csv","LapAndWeather/LapWeather_Singapore Grand Prix.csv","LapAndWeather/LapWeather_Spanish Grand Prix.csv","LapAndWeather/LapWeather_United States Grand Prix.csv"]
combined_csv = pd.concat([pd.read_csv(f) for f in filenames ])
# print(combined_csv)
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [4]:
df = pd.read_csv('combined_csv.csv')
df = df[df["IsAccurate"]==True]
df = df[df["TrackStatus"]==1]
df = df.drop(['Time','DriverNumber','LapStartDate','Stint', 'WindDirection','Unnamed: 0', 'PitOutTime', 'PitInTime', 'Sector1Time','Sector2Time','Sector3Time','Sector1SessionTime','Sector2SessionTime','Sector3SessionTime','SpeedI1','SpeedI2','SpeedST','IsPersonalBest','FreshTyre','SpeedFL','LapStartTime','Driver','TrackStatus','IsAccurate'], axis=1)
df = df.dropna()

dfOutput = df['LapTime']
train_targets = dfOutput.to_numpy()

for i in range(len(train_targets)):
    train_targets[i] = train_targets[i].replace('0 days ', '')
# print(train_targets)
actual_train_targets = []
for time in train_targets:
    td = parse(time) - parse('00:00:00')
    seconds = td.total_seconds()
    actual_train_targets.append(seconds)
# print(actual_train_targets)
dfLapTime = pd.DataFrame(actual_train_targets)

df['LapTime'] = dfLapTime
df = df.dropna()
cols = ['LapTime']
for i in range(1,20):
    print(len(df))
    Q1 = df.loc[df["Track"]==i][cols].quantile(0.25)
    Q3 = df.loc[df["Track"]==i][cols].quantile(0.75)
    IQR = Q3-Q1
    indexes = df.loc[df["Track"]==i][((df[cols] < (Q1 - 1.5 * IQR)) |(df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)].index
    df.drop(indexes,inplace=True)
    
dfInput = df.drop(['LapTime'], axis=1)
dfInput = dfInput.replace({'SUPERHARD':1, 'HARD':2, 'MEDIUM':3, 'SOFT':4,'SUPERSOFT': 5, 'ULTRASOFT': 6,'HYPERSOFT':7,"INTERMEDIATE":0,"WET":-1})
dfInput = dfInput.replace({'Ferrari':1,'Mercedes':2,'Red Bull Racing':3, 'McLaren':4, 'Renault':5, 'Force India':6, 'Sauber':7, 'Williams':8, 'Toro Rosso':9, 'Haas F1 Team':10, 'Racing Point':6})
train_inputs = dfInput.to_numpy()
inputs_array = train_inputs.astype('float64')
dfOutput = df['LapTime']

print(df)
targets_array = dfOutput.to_numpy()
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

new_shape = (len(targets_array), 1)
targets = targets.view(new_shape)
print(targets.shape)

print(inputs_array)
print(targets_array)


13743
13743
13743
13633
13633
13552
13552
13429
13429
13417
13401
13401
13388
13364
13196
13064
13064
13064
13064
       LapTime  LapNumber   Compound  TyreLife        Team  AirTemp  Humidity  \
1       90.177          2  ULTRASOFT       4.0     Ferrari     24.2      36.3   
2       89.610          3  ULTRASOFT       5.0     Ferrari     23.9      36.5   
3       89.540          4  ULTRASOFT       6.0     Ferrari     23.9      36.3   
4       89.481          5  ULTRASOFT       7.0     Ferrari     23.5      36.3   
7       88.405          8  ULTRASOFT      10.0     Ferrari     23.8      35.6   
...        ...        ...        ...       ...         ...      ...       ...   
17253  100.578         63  ULTRASOFT      50.0  Toro Rosso     26.2      57.1   
17254  100.566         64  ULTRASOFT      51.0  Toro Rosso     26.2      57.0   
17255  100.552         65  ULTRASOFT      52.0  Toro Rosso     26.2      57.0   
17256  100.830         66  ULTRASOFT      53.0  Toro Rosso     26.1      57.

C:\Users\helen\AppData\Local\Temp/ipykernel_12136/1073215378.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  indexes = df.loc[df["Track"]==i][((df[cols] < (Q1 - 1.5 * IQR)) |(df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)].index


In [5]:
dataset = TensorDataset(inputs, targets)

In [6]:
val_percent = 0.2
num_rows = len(dfInput.index)
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

In [7]:
batch_size = 32
train_loader = DataLoader(train_ds, batch_size, shuffle = True, num_workers = 0)
val_loader = DataLoader(val_ds, batch_size, num_workers = 0)

In [8]:
input_size = len(inputs[0])
output_size = 1

In [9]:
class MultiLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(11, 144),
            nn.ReLU(),
            nn.Linear(144,72),
            nn.ReLU(),
            nn.Linear(72,36),
            nn.ReLU(),
            nn.Linear(36,1)
        )
        
    def forward(self, xb):
        out = self.layers(xb)                       
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        mse_loss = nn.MSELoss()
        # cross_entropy_loss = nn.CrossEntropyLoss()
        # loss = cross_entropy_loss(out, targets)
        loss = mse_loss(out, targets)  
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        mse_loss = nn.MSELoss()
        loss = mse_loss(out, targets)    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [10]:
model = MultiLinearRegressionModel()

In [11]:
# list(model.parameters())

In [12]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [13]:
result = evaluate(model, val_loader)
print(result)

{'val_loss': 9022.494140625}


In [14]:
epochs = 1500
lr = 1e-6
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 181.3377
Epoch [40], val_loss: 176.2751
Epoch [60], val_loss: 186.3605
Epoch [80], val_loss: 195.5441
Epoch [100], val_loss: 165.4651
Epoch [120], val_loss: 160.7414
Epoch [140], val_loss: 156.4713
Epoch [160], val_loss: 174.8457
Epoch [180], val_loss: 145.8118
Epoch [200], val_loss: 143.7206
Epoch [220], val_loss: 131.6835
Epoch [240], val_loss: 120.0602
Epoch [260], val_loss: 151.5976
Epoch [280], val_loss: 127.2915
Epoch [300], val_loss: 135.6454
Epoch [320], val_loss: 112.7774
Epoch [340], val_loss: 103.0408
Epoch [360], val_loss: 103.8406
Epoch [380], val_loss: 106.0595
Epoch [400], val_loss: 98.3570
Epoch [420], val_loss: 103.2561
Epoch [440], val_loss: 100.0615
Epoch [460], val_loss: 97.1386
Epoch [480], val_loss: 97.5711
Epoch [500], val_loss: 100.1755
Epoch [520], val_loss: 96.2916
Epoch [540], val_loss: 97.7658
Epoch [560], val_loss: 95.8342
Epoch [580], val_loss: 93.5943
Epoch [600], val_loss: 101.8846
Epoch [620], val_loss: 100.6936
Epoch [640], val_lo

In [15]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                
    prediction = predictions[0].detach()
    return target, prediction

In [16]:
input, target = val_ds[12]
predict_single(input, target, model)

(tensor([81.7880]), tensor([80.7008]))

In [17]:
input, target = val_ds[50]
predict_single(input, target, model)

(tensor([82.1950]), tensor([101.2538]))

In [18]:
def get_accuracy(predict,actual):
        predict = predict.detach().numpy()
        actual = actual.numpy()

        count = 0

        for i in range(len(actual)):
            if predict[i] >= actual[i]*.9 and predict[i] <= actual[i]*1.1:
                count+=1
        return count/len(predict)*100


In [19]:
targets = []
predictions = []
for i in range(len(val_ds)):
    input, target = val_ds[i]
    target, prediction = predict_single(input, target, model)
    # np.append(targets,target)
    targets.append(target)
    predictions.append(prediction)
    # np.append(predictions,prediction)

# print(len(predictions))
# print(targets[0])
accuracy = []
for j in range(len(predictions)-1):
    accuracy.append(get_accuracy(predictions[j],targets[j]))

In [20]:
print("Within 10% of actual lap time:")
print(np.sum(accuracy)/len(accuracy))

Within 10% of actual lap time:
74.87552661815397


In [21]:
# fig, ax = plt.subplots(figsize=(20, 10))

# ax.scatter(range(len(predictions)),targets)
# ax.scatter(range(len(predictions)), predictions)
